#### 필요한 라이브러리 임포트

In [1]:
import time
from selenium import webdriver
import pandas as pd
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import re

#### Naver 로그인

In [ ]:
url='https://nid.naver.com/nidlogin.login'
id_ = ''
pw = ''
    
browser = webdriver.Chrome()
browser.get(url)

browser.implicitly_wait(2)

# Naver login 네이버 로그인
browser.execute_script("document.getElementsByName('id')[0].value=\'"+ id_ + "\'")
browser.execute_script("document.getElementsByName('pw')[0].value=\'"+ pw + "\'")
browser.find_element(by=By.XPATH,value='//*[@id="log.login"]').click()
time.sleep(1)

#### 키워드 하나에 대해서 csv파일 만들기

##### 카페에 들어가 원하는 키워드 입력 후 게시글의 링크와 제목 가져오기

In [ ]:
baseurl='https://cafe.naver.com/directwedding'
browser.get(baseurl)

keyword = '"lg" "스타일러"'
search_box = browser.find_element(By.ID, "topLayerQueryInput")
search_box.send_keys(keyword)
search_box.send_keys(Keys.ENTER)
time.sleep(1)
browser.switch_to.frame("cafe_main")

# "50개씩" 보기 옵션을 선택하기 위한 드롭다운 메뉴 클릭
dropdown_menu = browser.find_element(By.ID, "listSizeSelectDiv")
dropdown_menu.click()
time.sleep(1)

# "50개씩" 옵션 선택
fifty_option = browser.find_element(
    By.XPATH, "//a[contains(text(), '50개씩')]")
fifty_option.click()
time.sleep(1)

# 한번에 보이는 게시물수를 변경하면 "lg스타일러"가 /"lg스타일러/"로 변함
# 따라서 다시 검색..
search_box = browser.find_element(By.ID, "queryTop")
search_box.clear()
search_box.send_keys(keyword)
time.sleep(1)

# 후기(가전) 게시판 선택
board = browser.find_element(By.ID, "divSearchMenuTop")
board.click()
time.sleep(1)

board.option = browser.find_element(
    By.XPATH, "//a[contains(text(), '후기(가전)')]")
board.option.click()
time.sleep(1)

# 검색 옵션 드롭다운 메뉴 클릭
search_option_dropdown = browser.find_element(By.ID, "divSearchByTop")
search_option_dropdown.click()
time.sleep(1)

# '제목만' 옵션 선택
title_only_option = browser.find_element(
    By.XPATH, "//a[contains(text(), '제목만')]")
title_only_option.click()
time.sleep(1)

# 검색 버튼 클릭
search_btn = browser.find_element(
    By.XPATH, "//button[contains(text(), '검색')]")
search_btn.click()
time.sleep(1)

titles = []
links = []

while True:
    # BeautifulSoup으로 HTML을 파싱
    soup = bs(browser.page_source, 'html.parser')
    articles = soup.select('div.inner_list a.article')

    for article in articles:
        title = article.text.strip()
        link = article['href']
        titles.append(title)
        links.append(link)

    page_num = browser.find_element(By.CSS_SELECTOR, "a.on").text
    print(f"{page_num} 페이지에서 게시글 {len(articles)}개 수집 완료. 총 수집: {len(titles)}개")

    # 다음 페이지 버튼 클릭
    try:
        # 현재 페이지 버튼 찾기
        current_page = browser.find_element(By.CSS_SELECTOR, "a.on")
        next_page = current_page.find_element(By.XPATH, "following-sibling::a")
        next_page.click()
        time.sleep(2)
    except Exception:
        print("더 이상 이동할 페이지가 없습니다. 크롤링 종료.")
        break

# 결과 출력
print(f"총 수집된 게시글 수: {len(titles)}")

1 페이지에서 게시글 50개 수집 완료. 총 수집: 50개
2 페이지에서 게시글 50개 수집 완료. 총 수집: 100개
3 페이지에서 게시글 50개 수집 완료. 총 수집: 150개
4 페이지에서 게시글 50개 수집 완료. 총 수집: 200개
5 페이지에서 게시글 50개 수집 완료. 총 수집: 250개
6 페이지에서 게시글 50개 수집 완료. 총 수집: 300개
7 페이지에서 게시글 50개 수집 완료. 총 수집: 350개
8 페이지에서 게시글 44개 수집 완료. 총 수집: 394개
더 이상 이동할 페이지가 없습니다. 크롤링 종료.
총 수집된 게시글 수: 394


##### 링크 따라서 본문 크롤링 후 csv파일로 저장

In [ ]:
results = []
for title, link in zip(titles, links):
    retries = 1 # 해당 글을 몇번 들어갈건지 선정
    success = False # 글 접속 성공여부

    while retries > 0 and not success:
        # 게시글의 링크로 이동
        browser.get('https://cafe.naver.com/directwedding' + link)
        browser.switch_to.frame("cafe_main")
        page_source = browser.page_source
        soup_article = bs(page_source, 'html.parser')
        # Date
        date = soup_article.find('div', class_='tit-box').find('td', class_='m-tcol-c date').text.strip()
        # Nickname
        nickname = soup_article.find('td', class_='p-nick').text.strip()
        # content
        lines = soup_article.find('div', class_='tbody m-tcol-c').stripped_strings
        content = ' '.join(lines)

        # html태그제거 및 텍스트 다듬기
        pattern1 = '<[^>]*>'
        pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
        content = re.sub(pattern=pattern1, repl='', string=content)
        content = content.replace(pattern2, '')
        content = content.replace('\n', '')
        content = content.replace('\u200b', '')

        retries -= 1  # 재시도 횟수 감소

    results.append({
        'Keyword': keyword,
        'Title': title,
        'Date': date,
        'Link': 'https://cafe.naver.com/directwedding' + link,
        'Content': content
    })

df = pd.DataFrame(data=results)
df.to_csv("lg스타일러.csv", index=False, encoding="utf-8-sig")

#### 여러 키워드 검색결과를 하나의 csv파일로 저장

In [ ]:
# wanted naver cafe url
baseurl='https://cafe.naver.com/directwedding'
browser.get(baseurl)

# wanted keyword list
Search_Keyword = ['"무선스틱청소기"', '"무선청소기"', '"lg" "무선청소기"', '"삼성" "무선청소기"', '"다이슨" "무선청소기"', '"로봇청소기"']

results = []
keyword_list = []
titles_list = []
links_list = []

for keyword in Search_Keyword:
    search_box = browser.find_element(By.ID, "topLayerQueryInput")
    search_box.send_keys(keyword)
    search_box.send_keys(Keys.ENTER)
    time.sleep(1)
    browser.switch_to.frame("cafe_main")

    # "50개씩" 보기 옵션을 선택하기 위한 드롭다운 메뉴 클릭
    dropdown_menu = browser.find_element(By.ID, "listSizeSelectDiv")
    dropdown_menu.click()
    time.sleep(1)

    # "50개씩" 옵션 선택
    fifty_option = browser.find_element(By.XPATH, "//a[contains(text(), '50개씩')]")
    fifty_option.click()
    time.sleep(1)

    # 한번에 보이는 게시물수를 변경하면 "lg스타일러"가 /"lg스타일러/"로 변함
    # 따라서 다시 검색..
    search_box = browser.find_element(By.ID, "queryTop")
    search_box.clear()
    search_box.send_keys(keyword)
    time.sleep(1)

    # 후기(가전) 게시판 선택
    board = browser.find_element(By.ID, "divSearchMenuTop")
    board.click()
    time.sleep(1)

    board.option = browser.find_element(By.XPATH, "//a[contains(text(), '후기(가전)')]")
    board.option.click()
    time.sleep(1)

    # 검색 옵션 드롭다운 메뉴 클릭
    search_option_dropdown = browser.find_element(By.ID, "divSearchByTop")
    search_option_dropdown.click()
    time.sleep(1)

    # '제목만' 옵션 선택
    title_only_option = browser.find_element(By.XPATH, "//a[contains(text(), '제목만')]")
    title_only_option.click()
    time.sleep(1)

    # 검색 버튼 클릭
    search_btn = browser.find_element(By.XPATH, "//button[contains(text(), '검색')]")
    search_btn.click()
    time.sleep(1)

    titles = []
    links = []

    print(f'{keyword} 크롤링 시작')

    while True:
        # BeautifulSoup으로 HTML을 파싱
        soup = bs(browser.page_source, 'html.parser')
        articles = soup.select('div.inner_list a.article')

        for article in articles:
            title = article.text.strip()
            link = article['href']
            keyword_list.append(keyword)
            titles.append(title)
            links.append(link)

        page_num = browser.find_element(By.CSS_SELECTOR, "a.on").text
        print(f"{page_num} 페이지에서 게시글 {len(articles)}개 수집 완료. 총 수집: {len(titles)}개")

        # 다음 페이지 버튼 클릭
        try:
            # 현재 페이지 버튼 찾기
            current_page = browser.find_element(By.CSS_SELECTOR, "a.on")
            next_page = current_page.find_element(By.XPATH, "following-sibling::a")
            next_page.click()
            time.sleep(2)
        except Exception:
            print("더 이상 이동할 페이지가 없습니다. 크롤링 종료.")
            break

    # 결과 출력
    print(f"{keyword} 총 수집된 게시글 수: {len(titles)}\n")
    titles_list.extend(titles)
    links_list.extend(links)

    browser.get(baseurl)

# 본문 크롤링
for keyword, title, link in zip(keyword_list, titles_list, links_list):
    retries = 1 # 해당 글을 몇번 들어갈건지 선정
    success = False # 글 접속 성공여부

    while retries > 0 and not success:
        # 게시글의 링크로 이동
        browser.get('https://cafe.naver.com/directwedding' + link)
        browser.switch_to.frame("cafe_main")
        page_source = browser.page_source
        soup_article = bs(page_source, 'html.parser')
        # Date
        date = soup_article.find('div', class_='tit-box').find('td', class_='m-tcol-c date').text.strip()
        # Nickname
        nickname = soup_article.find('td', class_='p-nick').text.strip()
        # content
        lines = soup_article.find('div', class_='tbody m-tcol-c').stripped_strings
        content = ' '.join(lines)

        # html태그제거 및 텍스트 다듬기
        pattern1 = '<[^>]*>'
        pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
        content = re.sub(pattern=pattern1, repl='', string=content)
        content = content.replace(pattern2, '')
        content = content.replace('\n', '')
        content = content.replace('\u200b', '')

        retries -= 1  # 재시도 횟수 감소

    keyword = keyword.replace('"', '')
    results.append({
        'Keyword': keyword,
        'Title': title,
        'Date': date,
        'Link': 'https://cafe.naver.com/directwedding' + link,
        'Content': content
    })

df = pd.DataFrame(data=results)
df.to_csv("naver_cafe.csv", index=False, encoding="utf-8-sig")

"무선스틱청소기" 크롤링 시작
1 페이지에서 게시글 6개 수집 완료. 총 수집: 6개
더 이상 이동할 페이지가 없습니다. 크롤링 종료.
"무선스틱청소기" 총 수집된 게시글 수: 6

"무선청소기" 크롤링 시작
1 페이지에서 게시글 50개 수집 완료. 총 수집: 50개
2 페이지에서 게시글 50개 수집 완료. 총 수집: 100개
3 페이지에서 게시글 50개 수집 완료. 총 수집: 150개
4 페이지에서 게시글 50개 수집 완료. 총 수집: 200개
5 페이지에서 게시글 50개 수집 완료. 총 수집: 250개
6 페이지에서 게시글 50개 수집 완료. 총 수집: 300개
7 페이지에서 게시글 50개 수집 완료. 총 수집: 350개
8 페이지에서 게시글 50개 수집 완료. 총 수집: 400개
9 페이지에서 게시글 49개 수집 완료. 총 수집: 449개
10 페이지에서 게시글 50개 수집 완료. 총 수집: 499개
11 페이지에서 게시글 50개 수집 완료. 총 수집: 549개
12 페이지에서 게시글 50개 수집 완료. 총 수집: 599개
13 페이지에서 게시글 32개 수집 완료. 총 수집: 631개
더 이상 이동할 페이지가 없습니다. 크롤링 종료.
"무선청소기" 총 수집된 게시글 수: 631

"lg" "무선청소기" 크롤링 시작
1 페이지에서 게시글 50개 수집 완료. 총 수집: 50개
2 페이지에서 게시글 50개 수집 완료. 총 수집: 100개
3 페이지에서 게시글 41개 수집 완료. 총 수집: 141개
더 이상 이동할 페이지가 없습니다. 크롤링 종료.
"lg" "무선청소기" 총 수집된 게시글 수: 141

"삼성" "무선청소기" 크롤링 시작
1 페이지에서 게시글 50개 수집 완료. 총 수집: 50개
2 페이지에서 게시글 50개 수집 완료. 총 수집: 100개
3 페이지에서 게시글 24개 수집 완료. 총 수집: 124개
더 이상 이동할 페이지가 없습니다. 크롤링 종료.
"삼성" "무선청소기" 총 수집된 게시글 수: 124

"다이슨" "무선청소기" 크롤링 시작
1 페

#### AI모델 게시글 분리  
일반모델은 general_df로, AI모델은 ai_df로 분리

In [29]:
general_df = df[~df['Content'].str.contains('AI', case=False, na=False)]
ai_df = df[df['Content'].str.contains('AI', case=False, na=False)]
ai_df['Keyword'] = 'AI ' + ai_df['Keyword']

C:\Users\yjlee\AppData\Local\Temp\ipykernel_10040\2789329052.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ai_df['Keyword'] = 'AI ' + ai_df['Keyword']


In [32]:
print("일반모델:", general_df.shape[0])
print("AI모델:", ai_df.shape[0])
print("총 개수:", df.shape[0])

print("\n무선스틱청소기")
print((general_df['Keyword'].str.contains('무선스틱청소기')).sum())
print((ai_df['Keyword'].str.contains('무선스틱청소기')).sum())
print((df['Keyword'].str.contains('무선스틱청소기')).sum())

print("\n무선청소기")
print((general_df['Keyword'].str.contains('무선청소기')).sum())
print((ai_df['Keyword'].str.contains('무선청소기')).sum())
print((df['Keyword'].str.contains('무선청소기')).sum())

print("\nlg 무선청소기")
print((general_df['Keyword'].str.contains('lg 무선청소기')).sum())
print((ai_df['Keyword'].str.contains('lg 무선청소기')).sum())
print((df['Keyword'].str.contains('lg 무선청소기')).sum())

print("\n삼성 무선청소기")
print((general_df['Keyword'].str.contains('삼성 무선청소기')).sum())
print((ai_df['Keyword'].str.contains('삼성 무선청소기')).sum())
print((df['Keyword'].str.contains('삼성 무선청소기')).sum())

print("\n다이슨 무선청소기")
print((general_df['Keyword'].str.contains('다이슨 무선청소기')).sum())
print((ai_df['Keyword'].str.contains('다이슨 무선청소기')).sum())
print((df['Keyword'].str.contains('다이슨 무선청소기')).sum())

print("\n로봇청소기")
print((general_df['Keyword'].str.contains('로봇청소기')).sum())
print((ai_df['Keyword'].str.contains('로봇청소기')).sum())
print((df['Keyword'].str.contains('로봇청소기')).sum())

일반모델: 1842
AI모델: 103
총 개수: 1945

무선스틱청소기
6
0
6

무선청소기
892
36
928

lg 무선청소기
138
3
141

삼성 무선청소기
111
13
124

다이슨 무선청소기
32
0
32

로봇청소기
944
67
1011


In [31]:
general_df.to_csv("청소기.csv", index=False, encoding="utf-8-sig")
ai_df.to_csv("AI청소기.csv", index=False, encoding="utf-8-sig")